In [ ]:
from PIL import Image
import os
import numpy as np
import pandas as pd
import csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

In [ ]:
alex_images_path = "Alex_Kelly_Pics/Alex"
kelly_images_path = "Alex_Kelly_Pics/Kelly"
test_images_path = "Alex_Kelly_Pics/TestSet"
image_labels_path = "Kelly_and_Alex_Image_Labels - Sheet1.csv"
neither_images_path = "Neither_pics"

In [ ]:
def sorted_image_names(path):
    return sorted(os.listdir(path), key=lambda x: int(''.join(filter(str.isdigit, x))))

In [ ]:
alex_image_names = sorted_image_names(alex_images_path)
kelly_image_names = sorted_image_names(kelly_images_path)
test_image_names = sorted_image_names(test_images_path)

image_names = alex_image_names + kelly_image_names

In [ ]:
labels = pd.read_csv(image_labels_path)
le = LabelEncoder().fit(labels["Landscape"])
labels["Landscape"] = le.transform(labels["Landscape"])
labels

In [ ]:
labels['Photographer'] = labels['image_name'].str.split('-', n = 1).str[0]

In [ ]:
le = LabelEncoder().fit(labels["Photographer"])
labels["Photographer"] = le.transform(labels["Photographer"])

In [ ]:
labels

In [ ]:
def get_largest_size(folder):
    biggest_image = [0, 0]
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img.size[0] > biggest_image[0]: 
            biggest_image[0] = img.size[0]
        if img.size[1] > biggest_image[1]:
            biggest_image[1] = img.size[1]
    return biggest_image

In [ ]:
alex_im_size = get_largest_size(alex_images_path)
kelly_im_size = get_largest_size(kelly_images_path)
test_im_size = get_largest_size(test_images_path)

print("Max Image Sizes:", max(alex_im_size, kelly_im_size, test_im_size))

In [ ]:
def load_images_from_folder(folder, resize_shape=(700, 440)):
    images = []
    image_names = []
    count = 0
    for filename in os.listdir(folder):
        count += 1
        img = Image.open(os.path.join(folder, filename)).convert('RGB')
        img = img.resize(resize_shape)
        if img is not None:
            img_array = np.array(img)
            img_array = img_array.flatten()
            attr = labels[labels["image_name"] == filename].drop("image_name", axis = 1).to_numpy()
            img_array = np.append(img_array, attr)
            images.append(img_array)
            
        if count % int(len(os.listdir(folder)) / 10) == 0:
            print(f"{count / len(os.listdir(folder))*100}% Complete")
    return images

In [ ]:
alex_images = load_images_from_folder(alex_images_path)
kelly_images = load_images_from_folder(kelly_images_path)
test_images = load_images_from_folder(test_images_path)


In [ ]:
len(alex_images[0])

In [ ]:
test_images_array = np.array(test_images)
del(test_images)
test_images = test_images_array[:, 0:924000]
test_images = test_images / 255.0
test_images = test_images.reshape(test_images.shape[0], 700, 440, 3)

In [ ]:
merged_dataset = np.concatenate([alex_images, kelly_images])
del(alex_images, kelly_images) # Free up memory
merged_dataset

In [ ]:
num_attributes = labels.shape[1] - 1

attribute_index = list(range(924000, 924000+num_attributes))

attributes = [col for col in labels.columns if col != "image_name"]

attr_dict = dict(zip(attributes, attribute_index))

# CNN - Sunset

In [ ]:
X = merged_dataset[:, 0:924000]
X = X / 255.0
y = merged_dataset[:, attr_dict["Sunset"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = X_train.reshape(X_train.shape[0],700,440,3)
X_test = X_test.reshape(X_test.shape[0],700,440,3)

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
styles=[':','-.','--','-',':','-.','--','-',':','-.','--','-']

model = Sequential()
model.add(Conv2D(24,kernel_size=5,padding='same',activation='relu',
        input_shape=(700,440,3)))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs = 20, 
validation_data = (X_test, y_test), callbacks=[annealer], verbose=2)

plt.figure(figsize=(15,5))
plt.plot(history.history['val_accuracy'])
plt.title('Sunset model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
axes = plt.gca()
axes.set_ylim([0.60,1])
plt.show()

Sunset_classifications = model.predict(test_images)

Sunset_binary_classifications = (Sunset_classifications >= .5).astype(int)

Sunset_binary_classifications

# CNN - Flower

In [ ]:
X = merged_dataset[:, 0:924000]
X = X / 255.0
y = merged_dataset[:, attr_dict["Flower"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = X_train.reshape(X_train.shape[0],700,440,3)
X_test = X_test.reshape(X_test.shape[0],700,440,3)

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
styles=[':','-.','--','-',':','-.','--','-',':','-.','--','-']

In [ ]:
model = Sequential()
model.add(Conv2D(24,kernel_size=5,padding='same',activation='relu',
        input_shape=(700,440,3)))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs = 20, 
validation_data = (X_test, y_test), callbacks=[annealer], verbose=2)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['val_accuracy'])
plt.title('Flower model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
axes = plt.gca()
axes.set_ylim([0.60,1])
plt.show()

In [ ]:
Flower_classifications = model.predict(test_images)

threshold = .5
Flower_binary_classifications = (Flower_classifications > threshold).astype(int)

Flower_binary_classifications

# CNN - Animal

In [ ]:
X = merged_dataset[:, 0:924000]
X = X / 255.0
y = merged_dataset[:, attr_dict["Animal"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = X_train.reshape(X_train.shape[0],700,440,3)
X_test = X_test.reshape(X_test.shape[0],700,440,3)

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
styles=[':','-.','--','-',':','-.','--','-',':','-.','--','-']

model = Sequential()
model.add(Conv2D(24,kernel_size=5,padding='same',activation='relu',
        input_shape=(700,440,3)))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs = 20, 
validation_data = (X_test, y_test), callbacks=[annealer], verbose=2)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['val_accuracy'])
plt.title('Animal model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
axes = plt.gca()
axes.set_ylim([0.60,1])
plt.show()

In [ ]:
Animal_classifications = model.predict(test_images)

threshold = .5
Animal_binary_classifications = (Animal_classifications > threshold).astype(int)

Animal_binary_classifications

# CNN - Building

In [ ]:
X = merged_dataset[:, 0:924000]
X = X / 255.0
y = merged_dataset[:, attr_dict["Building"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = X_train.reshape(X_train.shape[0],700,440,3)
X_test = X_test.reshape(X_test.shape[0],700,440,3)

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
styles=[':','-.','--','-',':','-.','--','-',':','-.','--','-']

model = Sequential()
model.add(Conv2D(24,kernel_size=5,padding='same',activation='relu',
        input_shape=(700,440,3)))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs = 20, 
validation_data = (X_test, y_test), callbacks=[annealer], verbose=2)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['val_accuracy'])
plt.title('Building model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
axes = plt.gca()
axes.set_ylim([0.60,1])
plt.show()

In [ ]:
Building_classifications = model.predict(test_images)

threshold = .5
Building_binary_classifications = (Building_classifications > threshold).astype(int)

Building_binary_classifications

# CNN - Mask



In [ ]:
X = merged_dataset[:, 0:924000]
X = X / 255.0
y = merged_dataset[:, attr_dict["Building"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = X_train.reshape(X_train.shape[0],700,440,3)
X_test = X_test.reshape(X_test.shape[0],700,440,3)

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
styles=[':','-.','--','-',':','-.','--','-',':','-.','--','-']

model = Sequential()
model.add(Conv2D(24,kernel_size=5,padding='same',activation='relu',
        input_shape=(700,440,3)))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs = 20, 
validation_data = (X_test, y_test), callbacks=[annealer], verbose=2)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['val_accuracy'])
plt.title('Building model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
axes = plt.gca()
axes.set_ylim([0.60,1])
plt.show()

In [ ]:
Mask_classifications = model.predict(test_images)

threshold = .5
Mask_binary_classifications = (Mask_classifications > threshold).astype(int)

Mask_binary_classifications

# CNN - Gray-Hair

In [ ]:
X = merged_dataset[:, 0:924000]
X = X / 255.0
y = merged_dataset[:, attr_dict["Gray-Hair"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = X_train.reshape(X_train.shape[0],700,440,3)
X_test = X_test.reshape(X_test.shape[0],700,440,3)

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
styles=[':','-.','--','-',':','-.','--','-',':','-.','--','-']

model = Sequential()
model.add(Conv2D(24,kernel_size=5,padding='same',activation='relu',
        input_shape=(700,440,3)))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs = 20, 
validation_data = (X_test, y_test), callbacks=[annealer], verbose=2)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['val_accuracy'])
plt.title('Gray-hair model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
axes = plt.gca()
axes.set_ylim([0.60,1])
plt.show()

In [ ]:
gray_classifications = model.predict(test_images)

threshold = .5
gray_binary_classifications = (gray_classifications > threshold).astype(int)

gray_binary_classifications

# CNN - Fire

In [ ]:
X = merged_dataset[:, 0:924000]
X = X / 255.0
y = merged_dataset[:, attr_dict["Fire"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = X_train.reshape(X_train.shape[0],700,440,3)
X_test = X_test.reshape(X_test.shape[0],700,440,3)

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
styles=[':','-.','--','-',':','-.','--','-',':','-.','--','-']

model = Sequential()
model.add(Conv2D(24,kernel_size=5,padding='same',activation='relu',
        input_shape=(700,440,3)))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs = 20, 
validation_data = (X_test, y_test), callbacks=[annealer], verbose=2)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['val_accuracy'])
plt.title('Fire model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
axes = plt.gca()
axes.set_ylim([0.60,1])
plt.show()

In [ ]:
fire_classifications = model.predict(test_images)

threshold = .5
fire_binary_classifications = (fire_classifications > threshold).astype(int)

fire_binary_classifications

# CNN - Food_drink

In [ ]:
X = merged_dataset[:, 0:924000]
X = X / 255.0
y = merged_dataset[:, attr_dict["Food_drink"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = X_train.reshape(X_train.shape[0],700,440,3)
X_test = X_test.reshape(X_test.shape[0],700,440,3)

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
styles=[':','-.','--','-',':','-.','--','-',':','-.','--','-']

model = Sequential()
model.add(Conv2D(24,kernel_size=5,padding='same',activation='relu',
        input_shape=(700,440,3)))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs = 20, 
validation_data = (X_test, y_test), callbacks=[annealer], verbose=2)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['val_accuracy'])
plt.title('Food_drink model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
axes = plt.gca()
axes.set_ylim([0.60,1])
plt.show()

In [ ]:
food_classifications = model.predict(test_images)

threshold = .5
food_binary_classifications = (food_classifications > threshold).astype(int)

food_binary_classifications

# PART 2

Load in Dependencies for the Modeling.

In [1]:
from PIL import Image
import os
import numpy as np
import pandas as pd
import csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

Create variables to hold the file paths for each of the image types. We have one for the Alex images, one for the Kelly images, one for the test images, and one for the neither images.

In [2]:
alex_images_path = "Alex_Kelly_Pics/Alex"
kelly_images_path = "Alex_Kelly_Pics/Kelly"
test_images_path = "Alex_Kelly_Pics/TestSet"
neither_images_path = "Neither_pics"

This function below takes a folder of images and then converts the images in the folder to an RGB format. It also takes in the label for each image folder (Alex (0), Kelly (1), and Neither (2)) and appends the labels to the array. The output of this function is the images from a folder in RGB pixel format as well as the labels for the folder of images.

In [3]:
def load_images_from_folder(folder, label, resize_shape=(700, 440)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename)).convert('RGB')
        img = img.resize(resize_shape)
        if img is not None:
            img_array = np.array(img)
            images.append(img_array)
            labels.append(label)
    return images, labels

The code below uses the load_images_from_folder function to create labels and pixelated image values for each of the image folders. The all_images array combines the alex_images, kelly_images, and neither_images arrays. The all_labels array creates an array of all of the 3 labels categorized for the model classification. The all_images array is then divided by 255 so that all of the image values are between 0 and 1 (normalized).

In [4]:
alex_images, alex_labels = load_images_from_folder(alex_images_path, label=0)
kelly_images, kelly_labels = load_images_from_folder(kelly_images_path, label=1)
neither_images, neither_labels = load_images_from_folder(neither_images_path, label=2)

all_images = np.array(alex_images + kelly_images + neither_images)
all_labels = np.array(alex_labels + kelly_labels + neither_labels)
del(alex_images, kelly_images, neither_images)

all_labels = to_categorical(all_labels, num_classes=3)

all_images = all_images / 255.0

We then created a train test split with the all_images as the X and the all_labels as the y.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

The model below uses a 2d convolution layer with 24 filters. The Max Pool then reduces the spatial dimensions of the input layer. The flatten component flattens the previous layer. The first dense layer uses 256 neurons with reLu activation. The final output layer is a dense layer with 3 outputs using softmax activation. The model uses an adam optimizer and uses the categorical cross entropy loss function. It uses accuracy as the scoring metric. The model is then fit on the training data with a batch size of 32 and 20 epochs.

In [ ]:
model = Sequential()
model.add(Conv2D(24,kernel_size=5,padding='same',activation='relu',
        input_shape=(700,440,3)))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs = 20, validation_data = (X_test, y_test), callbacks=LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0), verbose=2)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

The function below (load_images_from_folder_2 takes a folder of images and a resize shape and then returns the images in an array from the folder in RGB pixel format.

In [ ]:
def load_images_from_folder_2(folder, resize_shape=(700, 440)):
    images = []
    image_names = []
    count = 0
    for filename in os.listdir(folder):
        count += 1
        img = Image.open(os.path.join(folder, filename)).convert('RGB')
        img = img.resize(resize_shape)
        if img is not None:
            img_array = np.array(img)
            img_array = img_array.flatten()
            images.append(img_array)
            
        if count % int(len(os.listdir(folder)) / 10) == 0:
            print(f"{count / len(os.listdir(folder))*100}% Complete")
    return images

The cell below uses the load_images_from_folder_2 function to load in the test_images. The test images are then made into an array in line 2. That array is then filtered to select all of the rows and only the first 924000 columns from the array in line 3. In line 4, all of the pixel integers are then converted to a number between 0 and 1 to normalize. The final step was to reshape the test_images to same format as the train images so that they can be used on our final model.

In [ ]:
test_images = load_images_from_folder_2(test_images_path)
test_images_array = np.array(test_images)
test_images = test_images_array[:, 0:924000]
test_images = test_images / 255.0
test_images = test_images.reshape(test_images.shape[0], 700, 440, 3)

The test_probs variable is a 3 column array that holds the probs that an image is of that class from 0 to 2. This is because the model created above is used on the test images. The test_preds variable then converts these probabilities to either a 0,1, or 2 based on which probability is highest in a given row.

In [ ]:
test_probs = model.predict(test_images)

test_preds = np.argmax(test_probs, axis=1)

The 0s, 1s, and 2s are then mapped back to the original names of Alex for 0, Kelly for 1, and Neither for 2.

In [ ]:
label_mapping = {
    0: 'Alex',
    1: 'Kelly',
    2: 'Neither'
}

test_image_names = [label_mapping[label] for label in test_preds]


test_image_names = np.array(test_image_names)

A dataframe is then created with the names of the images as a column and then the image classification as another column.

In [ ]:
image_n = [f'test-image-{i+1:02d}' for i in range(len(test_image_names))]

test_names_df = pd.DataFrame({
    'Test Image Name': image_n,
    'Classification': test_image_names
})
test_names_df